In [ ]:
import numpy as np

In [ ]:
def mvnorm(n, mu, Sigma, random_state=256):
    """
    Simulates n vectors from mvn(mu, Sigma)
    with mean mu and covariance matrix Sigma
    """
    np.random.seed(random_state)

    p = mu.size  # Get size of mu
    res = np.zeros((n, p))  # Initialize matrix to house our vectors as columns

    if n > 0 and p > 0:
        E = np.random.normal(size=n*p).reshape(n, p)
        res = ((E @ np.linalg.cholesky(Sigma) + mu).T).T
    
    return res

def wish(nu, Phi, random_state=256):
    """ 
    Simulates a wishart random matrix from wish(nu, Phi)
    """
    p = Phi.shape[0]
    Z = mvnorm(nu, np.zeros((p, )), Phi, random_state=random_state)
    return Z.T @ Z

In [ ]:
# Prior Parameters
n = X.shape[0]
p = X.shape[1]

mu0 = np.array([np.nanmean(X["Health Expenditure Per Capita"]),
                np.nanmean(X["Prevalence of Undernourishment"]),
                np.nanmean(X["Net Migration"]),
                np.nanmean(X["Prevalence of HIV"])])

sd0 = mu0/2  # Arbitrary, but gibbs sampler will converge regardless

L0 = np.full((p, p), 0.1)
np.fill_diagonal(L0, 1)
L0 = L0 * np.outer(sd0, sd0)

nu0 = p + 2
S0 = L0


# Starting Values
Sigma = S0
O = (X.notna()).astype(int)
X_full = X
X_full = X_full.fillna(X_full.mean())

# Gibbs Sampler
for s in range(0, 100):
    # Update Theta
    ybar = np.array(X_full.mean())
    Ln = np.linalg.inv(np.linalg.inv(L0) + n * np.linalg.inv(Sigma))
    mun = Ln @ ((np.linalg.inv(L0) @ mu0) + n * (np.linalg.inv(Sigma) @ ybar))
    theta = mvnorm(1, mun, Ln)

    # Update Sigma
    Sn = S0 + ((X_full - theta).T @ (X_full - theta))
    Sigma = np.linalg.inv(wish(nu0 + n, np.linalg.inv(Sn)))

    # Update Missing Data
    for i in range(0, n):
        missing = O.iloc[i,] == 0
        not_missing = O.iloc[i,] == 1
        b = [j for j, x in enumerate(missing) if x]
        a = [j for j, x in enumerate(not_missing) if x]

        iSa = np.linalg.inv(Sigma[np.ix_(a,a)])
        theta_b_given_a = theta[0,b] + (Sigma[np.ix_(b,a)] @ iSa @ (X_full.iloc[i, a] - theta[0,a]))
        Sigma_b_given_a = Sigma[np.ix_(b,b)] - (Sigma[np.ix_(b,a)] @ iSa @ Sigma[np.ix_(a,b)])

        X_full.iloc[i, b] = mvnorm(1, theta_b_given_a, Sigma_b_given_a).flatten()